In [ ]:
import numpy as np
import seaborn as sns
from scipy.integrate import solve_ivp
from SALib.sample import saltelli, sobol, fast_sampler
from SALib.analyze import sobol, fast
from scipy.constants import N_A
from scipy.optimize import fsolve
import math
import matplotlib.pyplot as plt
import SALib

In [ ]:
Target_cell_number = 2e5
well_size = 125e-6
sigma = well_size*N_A/Target_cell_number
A0s = np.geomspace(1e-13, 1e-4, 50)
t_end = 60*60*1000000
t = np.geomspace(1e-10, t_end, 20)
t_span = [1e-10, t_end]
z0 = [0, 0]
tumour_cell_radius = 8e-6
tumour_cell_surface_area = 4*math.pi*((tumour_cell_radius)**2)

In [ ]:
np.log10(sigma)

In [ ]:
def A1_steady_state(x, Ainit, k1, koff, k2, rtot):
    k1 = k1/sigma
    Atot = sigma*Ainit
    express = 2*k1*(rtot - x - 2*(k2*x*(rtot-x)/(2*(koff + k2*x))))*(Atot - x - (k2*x*(rtot-x)/(2*(koff + k2*x)))) -koff*x - k2*x*(rtot - x - 2*(k2*x*(rtot-x)/(2*(koff + k2*x)))) +2*koff*(k2*x*(rtot-x)/(2*(koff + k2*x)))

    return express

def A2_steady_state(x, k2, koff, rtot):
    express = k2*x*(rtot-x)/(2*(koff + k2*x))

    return express

def EC50_finder(array, A0s):
    half_max = 0.5*np.max(array) 
    half_max_array = half_max*np.ones_like(array)
    indicies = np.argwhere(np.diff(np.sign(half_max_array-array)))
    return A0s[indicies[0]]

def mono_valent_steady_state(Ainit, k1, koff, rtot):
    k1 = k1/sigma
    Atot = sigma*Ainit
    a = 1
    b = -((koff/k1) + rtot + Atot)
    c = rtot*Atot

    st = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

    return st

def mono_valent_steady_state_var(Ainit, KD, sigma, rtot):
    Atot = sigma*Ainit
    a = 1
    b = -((KD*sigma) + rtot + Atot)
    c = rtot*Atot

    st = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

    return st

def monovalent_binding(t, z, Ainit, rtot, kon, koff):
     k = kon/sigma
     Atot = well_size*N_A*Ainit/Target_cell_number
     A0 = Atot - z
     Ag = rtot - z

     dA1 = k*Ag*A0 - koff*z

     return dA1

In [ ]:
problem = {
    'num_vars': 4,
    'names': ['rtot', 'kon', 'D', 'koff'],
    'bounds': [[4, 6],
               [4, 6],
               [-15, -13],
               [-6, -4]]
}

problem1 = {
    'num_vars': 3,
    'names': ['rtot', 'KD', 'sigma'],
    'bounds': [[1e4, 1e6],
               [1e-12, 1e-8],
               [1e14, 1e15]]
}

problem2 = {
    'num_vars': 4,
    'names': ['rtot', 'kon', 'sigma', 'koff'],
    'bounds': [[4, 6],
               [4, 6],
               [1e14, 1e14+1],
               [-6, -4]]
}


vals = SALib.sample.sobol.sample(problem, 16384)
Y = np.zeros(len(vals))
Y1 = np.zeros(len(vals))

In [ ]:

for i, params in enumerate(vals):
    rtot = params[0]
    rtot = 10**rtot
    kon = params[1]
    kon = 10**kon
    D = params[2]
    D = 10**D
    koff = params[3]
    koff = 10**koff
    k2 = 4*D/tumour_cell_surface_area

    
    Ainit_array = np.zeros_like(A0s)
    Ainit_array1 = np.zeros_like(A0s)
    
    for j, A0 in enumerate(A0s):

        
        A1_st = fsolve(A1_steady_state, [0], args=(A0, kon, koff, k2, rtot))
        A2_st = A2_steady_state(A1_st, k2, koff, rtot)
        if (A1_st < 0) or (A2_st < 0) :
            print('negative')
        
        if ((A1_st + 2*A2_st) > rtot):
            print('blown up')

        Ab = A1_st + A2_st
        Ab1 = A1_st + 2*A2_st
        Ainit_array[j] = Ab[-1]
        Ainit_array1[j] = Ab1[-1]
    print(Ainit_array/rtot)
    
    biv_ec50 = EC50_finder(Ainit_array, A0s)
    biv_ec501 = EC50_finder(Ainit_array1, A0s)
    mono_ec50 = EC50_finder(mono_valent_steady_state(A0s, kon, koff, rtot), A0s)
    #Y[i] = mono_ec50
    #Y[i] = np.log10(mono_ec50/biv_ec50)
    Y1[i] = biv_ec501
    

In [ ]:
Si = sobol.analyze(problem, Y1, print_to_console=True)

In [ ]:
total, first, second = Si.to_df()
sns.set_context('talk')

In [ ]:
array = list(total['ST'].values)
array.append(0)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,6))
ax.bar(x =[r'$r^{tot}$', r'$k^{on}$', r'$D$', r'$k^{off}$', 'dummy'],  height=array, color=['purple', 'blue', 'skyblue', 'orange', 'black'])
ax.set_ylabel('Index Value')
ax.set_xlabel('Model Parameter')
fig.savefig('/Users/lukeheirene/ADCC-Project/Figures/bivalent_ec50_receptor_occupancy_total_order_sensiv_analysis.png')